In [1]:
import sys
!{sys.executable} -m pip install geopandas
!{sys.executable} -m pip install pyarrow # can maybe try pyarrow again
!{sys.executable} -m pip install rtree
!{sys.executable} -m pip install pygeos


In [2]:
USE_PYGEOS=1

In [3]:
import dask 
import rtree
import dask.dataframe as dd
import pandas as pd
import numpy as np
import pygeos
import geopandas as gpd
import os
gpd.options.use_pygeos = True
#import fastparquet
year = "yellow_tripdata_2010-01.csv"
shape = 'shape/taxi_zones.shp'
folder = r"taxi"
write = 'new_csvs'


In [4]:
shape_df = gpd.read_file('taxi_geo.json')
shape_df.drop(columns=['id', 'objectid', 'shape_leng', 'shape_area', 'zone','borough'],inplace=True)

In [5]:
list_a = [i.name for i in os.scandir(folder)]

In [6]:
'''this should be run as a .py script to avoid the kernel dieing.
Otherwise will need to run on each element in list_a'''

for i in os.scandir(folder):
    file = i.name
    df = pd.read_csv(folder + '/'+ file)
    
    gdf_listings = gpd.GeoDataFrame(df,   geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude))
    dfi = gpd.sjoin(gdf_listings, shape_df,how='left', op='within')
    dfi.rename(columns={'locationid':'pickup_location'},inplace=True)
    dfi.drop(columns='index_right',inplace=True)
    
    gdf_listings = gpd.GeoDataFrame(dfi, geometry=gpd.points_from_xy(dfi.dropoff_longitude, dfi.dropoff_latitude))
    final_df = gpd.sjoin(gdf_listings, shape_df,how='left', op='within')
    final_df.rename(columns={'locationid':'dropoff_location'},inplace=True)
    final_df.drop(columns=['index_right', 'id', 'objectid', 'shape_leng', 'shape_area','geometry'],inplace=True)
    
    final_df.to_csv(folder + '/zones_' + file)